# Image Classification

In [1]:
#MNIST image list
#set of handwritten characters from 0-9
#first download and save.

In [2]:
from urllib.request import urlretrieve
import sys
import gzip
import shutil
import os
import struct
import numpy as np

In [3]:
#2
# 0 0 1 0 0 0 0 0 

def savetxt(filename, ndarray):
    with open(filename, 'w') as f:
        labels = list(map(' '.join, np.eye(10, dtype=np.uint).astype(str)))
        for row in ndarray:
            row_str = row.astype(str)
            label_str = labels[row[-1]]
            feature_str = ' '.join(row_str[:-1])
            f.write('|labels {} |features {}\n'.format(label_str, feature_str))
            
def loadData(src, cimg):
    print ('Downloading ' + src)
    gzfname, h = urlretrieve(src, './delete.me')
    print ('Done.')
    try:
        with gzip.open(gzfname) as gz:
            n = struct.unpack('I', gz.read(4))
            # Read magic number.
            if n[0] != 0x3080000:
                raise Exception('Invalid file: unexpected magic number.')
            # Read number of entries.
            n = struct.unpack('>I', gz.read(4))[0]
            if n != cimg:
            
            
                raise Exception('Invalid file: expected {0} entries.'.format(cimg))
            crow = struct.unpack('>I', gz.read(4))[0]
            ccol = struct.unpack('>I', gz.read(4))[0]
            if crow != 28 or ccol != 28:
                raise Exception('Invalid file: expected 28 rows/cols per image.')
            # Read data.
            res = np.fromstring(gz.read(cimg * crow * ccol), dtype = np.uint8)
    finally:
        os.remove(gzfname)
    return res.reshape((cimg, crow * ccol))

def loadLabels(src, cimg):
    print ('Downloading ' + src)
    gzfname, h = urlretrieve(src, './delete.me')
    print ('Done.')
    try:
        with gzip.open(gzfname) as gz:
            n = struct.unpack('I', gz.read(4))
            # Read magic number.
            if n[0] != 0x1080000:
                raise Exception('Invalid file: unexpected magic number.')
            # Read number of entries.
            n = struct.unpack('>I', gz.read(4))
            if n[0] != cimg:
                raise Exception('Invalid file: expected {0} rows.'.format(cimg))
            # Read labels.
            res = np.fromstring(gz.read(cimg), dtype = np.uint8)
    finally:
        os.remove(gzfname)
    return res.reshape((cimg, 1))

In [4]:
def load(dataSrc, labelsSrc, cimg):
    data = loadData(dataSrc, cimg)
    labels = loadLabels(labelsSrc, cimg)
    return np.hstack((data, labels))

In [5]:
#Import dataset
train_path = './Train-28x28_cntk_text.txt'
test_path = './Test-28x28_cntk_text.txt'

train = load('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 60000)

print ('Writing train text file...')

savetxt(train_path, train)

print ('Done.')

test = load('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
    'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 10000)

print ('Writing test text file...')

savetxt(test_path, test)

print ('Done.')

Done.
Done.
Writing train text file...
Done.
Done.
Done.
Writing test text file...
Done.


In [6]:
import cntk as C
from cntk.train import Trainer, minibatch_size_schedule 
from cntk.io import MinibatchSource, CTFDeserializer, StreamDef, StreamDefs, INFINITELY_REPEAT
from cntk.device import cpu, try_set_default_device
from cntk.learners import adadelta, learning_rate_schedule, UnitType
from cntk.ops import relu, element_times, constant
from cntk.layers import Dense, Sequential, For
from cntk.losses import cross_entropy_with_softmax
from cntk.metrics import classification_error
from cntk.train.training_session import *
from cntk.logging import ProgressPrinter


In [7]:
28*28

784

In [11]:
input_dim = 784
num_output_classes=10
num_hidden_layers=1
hidden_layers_dim = 200

In [9]:
# Input variables denoting the features and label data
feature = C.input_variable(input_dim, np.float32)
label = C.input_variable(num_output_classes, np.float32)

In [12]:
scaled_input = element_times(constant(0.00390625), feature)

z = Sequential([For(range(num_hidden_layers), lambda i: Dense(hidden_layers_dim, activation=relu)),
                Dense(num_output_classes)])(scaled_input)

ce = cross_entropy_with_softmax(z, label)
pe = classification_error(z, label)

In [13]:
reader_train = MinibatchSource(CTFDeserializer(train_path, StreamDefs(
        features  = StreamDef(field='features', shape=input_dim, is_sparse=False),
        labels    = StreamDef(field='labels',   shape=num_output_classes, is_sparse=False)
    )), randomize=True, max_sweeps = INFINITELY_REPEAT)
    

In [14]:
input_map = {
    feature  : reader_train.streams.features,
    label  : reader_train.streams.labels
}

In [15]:
# Training config
minibatch_size = 64
num_samples_per_sweep = 60000
num_sweeps_to_train_with = 10


In [16]:
progress_writers = [ProgressPrinter(
    #freq=training_progress_output_freq,
    tag='Training',
    num_epochs=num_sweeps_to_train_with)]

In [18]:
lr = learning_rate_schedule(1, UnitType.sample)

trainer = Trainer(z, (ce, pe), adadelta(z.parameters, lr), progress_writers)


In [19]:
training_session(
    trainer=trainer,
    mb_source = reader_train,
    mb_size = minibatch_size,
    model_inputs_to_streams = input_map,
    max_samples = num_samples_per_sweep * num_sweeps_to_train_with,
    progress_frequency=num_samples_per_sweep
).train()

Learning rate per sample: 1.0
Finished Epoch[1 of 10]: [Training] loss = 0.382969 * 60032, metric = 10.22% * 60032 42.944s (1397.9 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.202870 * 59968, metric = 5.72% * 59968 1.525s (39323.3 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.156793 * 60032, metric = 4.44% * 60032 3.592s (16712.7 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.128566 * 59968, metric = 3.65% * 59968 1.702s (35233.8 samples/s);
Finished Epoch[5 of 10]: [Training] loss = 0.109716 * 60032, metric = 3.10% * 60032 1.704s (35230.0 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.095438 * 59968, metric = 2.69% * 59968 1.845s (32503.0 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.084263 * 60032, metric = 2.38% * 60032 1.852s (32414.7 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.075855 * 59968, metric = 2.11% * 59968 1.789s (33520.4 samples/s);
Finished Epoch[9 of 10]: [Training] loss = 0.068660 * 60032, metric = 1.9

In [32]:
reader_test =  MinibatchSource(CTFDeserializer(test_path, StreamDefs(
        features  = StreamDef(field='features', shape=input_dim, is_sparse=False),
        labels    = StreamDef(field='labels',   shape=num_output_classes, is_sparse=False)
    )), randomize=False, max_sweeps = 1)

input_map = {
    feature  : reader_test.streams.features,
    label  : reader_test.streams.labels
}


In [33]:
# Test data for trained model
test_minibatch_size = 1024
num_samples = 10000
num_minibatches_to_test = num_samples / test_minibatch_size
test_result = 0.0
for i in range(0, int(num_minibatches_to_test)):
    mb = reader_test.next_minibatch(test_minibatch_size, input_map=input_map)
    eval_error = trainer.test_minibatch(mb)
    test_result = test_result + eval_error

# Average of evaluation errors of all test minibatchese
eval = test_result / num_minibatches_to_test
print(eval)

0.0232


In [66]:
import scipy.ndimage
data = np.vectorize(lambda x: 255 - x)(np.ndarray.flatten(scipy.ndimage.imread("/home/nbuser/3.png", flatten=True)))

from cntk import load_model

#loaded_model = load_model("my", 'float')

pred = np.squeeze(z.eval({feature:[data]}))


top_class = np.argmax(pred)

top_class

/home/nbuser/anaconda3_431/lib/python3.6/site-packages/cntk/core.py:351: UserWarning: your data is of type "float64", but your input variable (uid "Input3") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


3